# Multi Model Building (Batch 1)

In [ ]:
FITTING_BATCH_NUM = 1

In [ ]:
# Remove TF logging warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
from model_helper_functions import create_fit_and_save_model, send_ifttt_notification, update_top_values
from data_pipeline import prep_data, get_train_val_test_data
from itertools import product
import re
import pathlib
import shutil

# Import correct optimizer
import platform
if platform.system() == "Darwin" and platform.processor() == "arm":
    from keras.optimizers.legacy import RMSprop, Adam
else:
    from keras.optimizers import RMSprop, Adam

In [ ]:
# Remove TF logging warnings
tf.get_logger().setLevel('ERROR')

In [ ]:
tf.random.set_seed(15)
keras.utils.set_random_seed(15)

In [ ]:
data_dir = '../bears'
df = prep_data(data_dir)
train_df, val_df, test_df = get_train_val_test_data(df)

In [ ]:
image_count = len(list(pathlib.Path(data_dir).glob('*/*')))
print('Total image count:',image_count)
print('Image count equal to dataframe length?', image_count == len(df))

In [ ]:
num_train_samples = len(train_df)
num_val_samples = len(val_df)
num_test_samples = len(test_df)
print('Number of training samples:',num_train_samples)
print('Number of validation samples:',num_val_samples)
print('Number of test samples:',num_test_samples)

---

## Hyperparameter Grid

In [ ]:
g_batch_size = [4, 8]
g_epochs = [20]
g_augmentation_params = [
    # either None or (flip, rotate_factor, zoom_factor, random_flip_str)
    None,
    (True, 0.25, 0.25, 'horizontal'),
]
g_cnn_params = [
    # cnn_units, cnn_filters, cnn_strides
    [(32,), [(3,3)], [(1,1)]],
    [(32, 64), [(5,5), (3,3)], [(2,2), (1,1)]],
]
g_dropout = [0, 0.5]
g_dense_units = [
    (32, 3),
    (64, 3),
]
g_activation = ['relu']
g_optimizer = [Adam]
g_earlystop_patience = [5, 10]
g_reducel_patience = [1]

In [ ]:
all_combinations = list(product(g_batch_size, g_epochs, g_augmentation_params, g_cnn_params, g_dropout,
                                g_dense_units, g_activation, g_optimizer, g_earlystop_patience, g_reducel_patience))
print(f'There are {len(all_combinations)} parameter combinations to run.')

In [ ]:
# Keep track of model fitting in order to resume at a later time if needed.
progress_file = f'./model_checkpoints_{FITTING_BATCH_NUM}/model_building_progress.csv'
os.makedirs(f'./model_checkpoints_{FITTING_BATCH_NUM}/', exist_ok=True)
if os.path.exists(progress_file):
    print('Using existing progress file.')
    progress = pd.read_csv(progress_file)
else:
    print('Creating new progress file.')
    progress = pd.DataFrame.from_records(all_combinations, columns=['batch_size', 'epochs', 'augmentation_params', 'cnn_params',
                                                                    'dropout', 'dense_units', 'activation', 'optimizer',
                                                                    'earlystop_patience', 'reducel_patience'])
    progress.insert(0, 'model', range(len(all_combinations)))
    progress['finished'] = False
    class_names = '(Adam|RMSprop)'
    progress['optimizer'] = progress['optimizer'].apply(lambda x: re.sub(f'.*{class_names}.*','\\1',str(x)))
    
    progress.to_csv(progress_file, index=False)
    progress = pd.read_csv(progress_file)

In [ ]:
display(progress)

---

## Fit Models

In [ ]:
os.makedirs(f'./model_checkpoints_{FITTING_BATCH_NUM}/val_metrics', exist_ok=True)
os.makedirs(f'./model_checkpoints_{FITTING_BATCH_NUM}/model_histories', exist_ok=True)
top_val_acc = dict()
models = []
for i, params in enumerate(all_combinations):
    if progress.at[i, 'finished']:
        print(f'Model {i} has already been fitted.')
    else:
        try:
            val_metrics = create_fit_and_save_model(f'model{i}', train_df, val_df, test_df, params, FITTING_BATCH_NUM)
            progress.at[i, 'finished'] = True
            progress.to_csv(progress_file, index=False)
            print(f"""Model {i}: validation accuracy= {val_metrics['accuracy']:.4f}, ran epochs= {val_metrics['ran_epochs']}, best epoch= {val_metrics['best_epoch']}, time={val_metrics['total_time']/60:.2f}min.""")
            
            update_top_values(top_val_acc, val_metrics['accuracy'], i, FITTING_BATCH_NUM)
                
        except:
            print(f'ERROR fitting model {i}')

In [ ]:
send_ifttt_notification('finished')

---